# Step 14: V11 — ModShift (signal uniqueness)

Goal: test whether there are significant transit-like signals at other phases besides the primary transit.

Why this matters:
- Eclipsing binaries (especially eccentric EBs) can produce strong secondary/tertiary events at unexpected phases.
- ModShift is a standard Kepler/robovetter-style uniqueness diagnostic.

This check requires the `exovetter` dependency.


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

ds = sh.load_dataset()
lc = sh.stitch_pdcsap(ds)
depth_ppm, _ = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

session = sh.make_session(stitched=lc, candidate=candidate, network=False)
r = session.run('V11')

print(
    json.dumps(
        {
            'status': r.status,
            'flags': r.flags,
            'metrics': r.metrics,
        },
        indent=2,
        sort_keys=True,
    )
)


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "flags": [],
  "metrics": {
    "baseline_days": 787.27,
    "false_alarm_threshold": 2.516974,
    "fred": 54.286904,
    "n_points": 73805,
    "n_transits_expected": 55,
    "positive_primary_ratio": 0.368431,
    "positive_signal": 2266.986951,
    "primary_signal": 6153.077969,
    "secondary_primary_ratio": 0.245483,
    "secondary_signal": 1510.476837,
    "tertiary_primary_ratio": 0.240508,
    "tertiary_signal": 1479.861607
  },
  "status": "ok"
}
```

</details>


In [ ]:
# Plot V11: ModShift
out = {
    'status': r.status,
    'flags': r.flags,
}

try:
    import matplotlib.pyplot as plt
    from bittr_tess_vetter.api import plot_modshift
    PLOTTING_AVAILABLE = True
except Exception as e:
    PLOTTING_AVAILABLE = False
    out['plotting_error'] = str(e)

if PLOTTING_AVAILABLE and (r.status == 'ok'):
    run_out_dir, docs_out_dir = sh.artifact_dirs(step_id='14_v11_modshift')
    run_path = run_out_dir / 'V11_modshift.png'
    docs_path = (docs_out_dir / 'V11_modshift.png') if docs_out_dir is not None else None

    fig, ax = plt.subplots(figsize=(8, 5))
    plot_modshift(r, ax=ax)
    ax.set_title('V11: ModShift (signal uniqueness)')
    fig.tight_layout()

    fig.savefig(run_path, dpi=150, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=150, bbox_inches='tight')
    plt.show()

    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


**Pre-rendered plot (no execution required):** `../artifacts/tutorial_toi-5807-incremental/14_v11_modshift/V11_modshift.png`

![V11: ModShift](../artifacts/tutorial_toi-5807-incremental/14_v11_modshift/V11_modshift.png)


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/14_v11_modshift/V11_modshift.png",
  "flags": [],
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/14_v11_modshift/V11_modshift.png",
  "status": "ok"
}
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Flags:** none.
- **Result:** primary peak at phase 0 with secondary peak at phase ∼0.354; secondary/primary ratio ≈0.245.
- **Interpretation:** non-primary structure is present at a non-0.5 phase; treat as a warning to corroborate with variability/robustness diagnostics.
- **Next step:** V12 (SWEET) to check sinusoidal variability at P and P/2; later, re-run V11/V12 on sector-selected or detrended light curves.

</details>
